In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def scrape_wait_times(url):
    """
    Scrapes a queue-times.com calendar page for average
    and maximum wait times per ride.
    Returns a DataFrame with columns: Date, Ride, Average Wait Time (mins), Max Wait Time (mins)
    """
    try:
        # 1. Fetch the webpage content
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Check for any request errors

        # 2. Parse the HTML with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # 3. Extract date from URL (format: /parks/16/calendar/2025/06/11)
        # or use current date as fallback
        url_parts = url.split('/')
        if len(url_parts) >= 7:
            year, month, day = url_parts[-3], url_parts[-2], url_parts[-1]
            date_str = f"{year}-{month}-{day}"
        else:
            date_str = datetime.now().strftime("%Y-%m-%d")

        # 4. Find all tables with class "table is-fullwidth"
        tables = soup.find_all('table', class_='table is-fullwidth')
        
        if not tables:
            print("Warning: Could not find any tables with class 'table is-fullwidth'")
            return pd.DataFrame()
        
        # 5. Create a dictionary to store data by ride name
        rides_data = {}
        
        for idx, table in enumerate(tables):
            print(f"\n--- Table {idx + 1} ---")
            
            # Find all rows in tbody
            tbody = table.find('tbody')
            if not tbody:
                print("Warning: Could not find tbody in table")
                continue
            
            rows = tbody.find_all('tr')
            
            # Extract ride names and wait times
            for row in rows:
                cols = row.find_all('td')
                if len(cols) >= 2:
                    # Get ride name from the first column (extract text from anchor tag if present)
                    ride_link = cols[0].find('a')
                    ride_name = ride_link.get_text(strip=True) if ride_link else cols[0].get_text(strip=True)
                    
                    # Get wait time from the second column (extract from span if present)
                    time_span = cols[1].find('span')
                    wait_time = time_span.get_text(strip=True) if time_span else cols[1].get_text(strip=True)
                    
                    # Initialize ride entry if not exists
                    if ride_name not in rides_data:
                        rides_data[ride_name] = {
                            'Date': date_str,
                            'Ride': ride_name,
                            'Average Wait Time (mins)': None,
                            'Max Wait Time (mins)': None
                        }
                    
                    # Assign to appropriate column based on table number
                    if idx == 0:  # Table 1 is Average
                        rides_data[ride_name]['Average Wait Time (mins)'] = float(wait_time)
                    elif idx == 1:  # Table 2 is Max
                        rides_data[ride_name]['Max Wait Time (mins)'] = float(wait_time)
                    
                    print(f"{ride_name}: {wait_time} mins")
        
        # 6. Convert dictionary to list of dictionaries for DataFrame
        data = list(rides_data.values())
        
        # 7. Create DataFrame
        df = pd.DataFrame(data)
        return df

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

# --- Main execution ---
if __name__ == "__main__":
    target_url = "https://queue-times.com/parks/16/calendar/2025/06/11"
    print(f"Scraping wait time forecast for: {target_url}\n")
    df = scrape_wait_times(target_url)
    print(f"\n--- DataFrame ---")
    print(df)

Scraping wait time forecast for: https://queue-times.com/parks/16/calendar/2025/06/11


--- Table 1 ---
Star Wars: Rise of the Resistance: 71 mins
Space Mountain: 62 mins
Tiana's Bayou Adventure: 61 mins
Indiana Jones™ Adventure: 53 mins
Big Thunder Mountain Railroad: 41 mins
Roger Rabbit's Car Toon Spin: 34 mins
Haunted Mansion Holiday: 33 mins
Mickey's House and Meet Mickey Mouse: 33 mins
Peter Pan's Flight: 32 mins
Millennium Falcon: Smugglers Run: 32 mins
Meet Disney Princesses at Royal Hall: 30 mins
Carnation Café: 29 mins
Cafe Orleans: 27 mins
Pirates of the Caribbean: 27 mins
Alice in Wonderland: 26 mins
Mickey & Minnie's Runaway Railway: 25 mins
Star Tours - The Adventures Continue: 23 mins
Oga's Cantina at the Disneyland Resort: 22 mins
Finding Nemo Submarine Voyage: 22 mins
Buzz Lightyear Astro Blasters: 21 mins
Dumbo the Flying Elephant: 20 mins
Autopia: 20 mins
Astro Orbitor: 19 mins
Jungle Cruise: 19 mins
Mr. Toad's Wild Ride: 17 mins
Snow White's Enchanted Wish: 15 mins
R

In [8]:
df

,Date,Ride,Average Wait Time (mins),Max Wait Time (mins)
0,2025-06-11,Star Wars: Rise of the Resistance,71.0,100.0
1,2025-06-11,Space Mountain,62.0,95.0
2,2025-06-11,Tiana's Bayou Adventure,61.0,120.0
3,2025-06-11,Indiana Jones™ Adventure,53.0,90.0
4,2025-06-11,Big Thunder Mountain Railroad,41.0,65.0
5,2025-06-11,Roger Rabbit's Car Toon Spin,34.0,65.0
6,2025-06-11,Haunted Mansion Holiday,33.0,65.0
7,2025-06-11,Mickey's House and Meet Mickey Mouse,33.0,60.0
8,2025-06-11,Peter Pan's Flight,32.0,60.0
9,2025-06-11,Millennium Falcon: Smugglers Run,32.0,115.0
